In [45]:
import scanpy
import scipy.sparse as sp
import os
import numpy as np
from scipy.spatial import Delaunay
import pandas as pd 
from sklearn.neighbors import NearestNeighbors

In [ ]:
f3000=scanpy.read_loom('/nfs/latdata/xinyi/slideseq/seuratresults/slideseq3000feature/slideseq3000feature_nograph.loom',sparse=True)


In [ ]:
f5000=scanpy.read_loom('/nfs/latdata/xinyi/slideseq/seuratresults/slideseq5000feature/slideseq5000feature.loom',sparse=True)

/home/xinyizxx/anaconda3/envs/conda_env/lib/python3.7/site-packages/loompy/loom_validator.py:48: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(path) as f:


In [34]:
f3000_coord=pd.read_csv('/nfs/latdata/xinyi/slideseq/seuratresults/slideseq3000feature/coordinates.csv')
f5000_coord=pd.read_csv('/nfs/latdata/xinyi/slideseq/seuratresults/slideseq5000feature/coordinates.csv')

In [5]:
def subsetX_varFeature(sobj,savepath=None):
    """
    param sobj: seurat object saved as loom, imported as anndata by scanpy
    """
    sobj_Xvar=sobj.X[:,np.nonzero(sobj.var.Selected.to_numpy())[0]]
    if savepath != None:
        sp.save_npz(savepath,sobj_Xvar)
    return sobj_Xvar

def subsetX_varSpatial(sobj,savepath=None):
    sobj_Xspvar=sobj.X[:,np.where(f3000.var.moransi_spatially_variable.to_numpy()==1)[0]]
    if savepath !=None:
        sp.save_npz(savepath,sobj_Xspvar)
    return sobj_Xspvar
def savefullX(sobj,savepath):
    sp.save_npz(savepath,sobj.X)

In [57]:
def getA_delaunay(sobj_coordMtx,savepath=None):
    a_size=sobj_coordMtx.shape[0]
    a=sp.lil_matrix((a_size,a_size))
    
    sobj_coord_np=sobj_coordMtx.loc[:,['x','y']].to_numpy()
    tri = Delaunay(sobj_coord_np)
    
    for tri_i in range(tri.simplices.shape[0]):
        tri_i_idx=tri.simplices[tri_i,:]
        a[tri_i_idx[0],tri_i_idx[1]]=1
        a[tri_i_idx[1],tri_i_idx[0]]=1
        a[tri_i_idx[0],tri_i_idx[2]]=1
        a[tri_i_idx[2],tri_i_idx[0]]=1
        a[tri_i_idx[1],tri_i_idx[2]]=1
        a[tri_i_idx[2],tri_i_idx[1]]=1
    
    a=a.tocsr()
    if savepath !=None:
        sp.save_npz(savepath,a)
    return a

def getA_knn(sobj_coordMtx,k,a_mode,savepath=None):
    sobj_coord_np=sobj_coordMtx.loc[:,['x','y']].to_numpy()
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(sobj_coord_np)
    a=nbrs.kneighbors_graph(sobj_coord_np,mode=a_mode)
    if a_mode=='connectivity':
        a=a-sp.identity(sobj_coordMtx.shape[0],format='csr')
    if savepath !=None:
        sp.save_npz(savepath,a)
    return a

In [20]:
xsavepath=os.path.join('/nfs','latdata','xinyi','slideseq','xamatrices','x')
f3000_XvarF=subsetX_varFeature(f3000,os.path.join(xsavepath,'f3000_XvarFeatures.npz'))
f3000_XspvarF=subsetX_varSpatial(f3000,os.path.join(xsavepath,'f3000_XvarFeaturesSpatial.npz'))
savefullX(f3000,os.path.join(xsavepath,'f3000_fullX.npz'))
f5000_XvarF=subsetX_varFeature(f5000,os.path.join(xsavepath,'f5000_XvarFeatures.npz'))
f5000_XspvarF=subsetX_varSpatial(f5000,os.path.join(xsavepath,'f5000_XvarFeaturesSpatial.npz'))


In [58]:
asavepath=os.path.join('/nfs','latdata','xinyi','slideseq','xamatrices','a')
f3000_a={}
f3000_a['f3000_a_delaunay']=getA_delaunay(f3000_coord,os.path.join(asavepath,'f3000_a_delaunay.npz'))
f3000_a['f3000_a_knn5_connectivity']=getA_knn(f3000_coord,5,'connectivity',os.path.join(asavepath,'f3000_knn5_connectivity.npz'))
f3000_a['f3000_a_knn5_distance']=getA_knn(f3000_coord,5,'distance',os.path.join(asavepath,'f3000_knn5_distance.npz'))
f3000_a['f3000_a_knn10_connectivity']=getA_knn(f3000_coord,10,'connectivity',os.path.join(asavepath,'f3000_knn10_connectivity.npz'))
f3000_a['f3000_a_knn10_distance']=getA_knn(f3000_coord,10,'distance',os.path.join(asavepath,'f3000_knn10_distance.npz'))
f3000_a['f3000_a_knn20_connectivity']=getA_knn(f3000_coord,20,'connectivity',os.path.join(asavepath,'f3000_knn20_connectivity.npz'))
f3000_a['f3000_a_knn20_distance']=getA_knn(f3000_coord,20,'distance',os.path.join(asavepath,'f3000_knn20_distance.npz'))

f5000_a={}
f5000_a['f5000_a_delaunay']=getA_delaunay(f5000_coord,os.path.join(asavepath,'f5000_a_delaunay.npz'))
f5000_a['f5000_a_knn5_connectivity']=getA_knn(f5000_coord,5,'connectivity',os.path.join(asavepath,'f5000_knn5_connectivity.npz'))
f5000_a['f5000_a_knn5_distance']=getA_knn(f5000_coord,5,'distance',os.path.join(asavepath,'f5000_knn5_distance.npz'))
f5000_a['f5000_a_knn10_connectivity']=getA_knn(f5000_coord,10,'connectivity',os.path.join(asavepath,'f5000_knn10_connectivity.npz'))
f5000_a['f5000_a_knn10_distance']=getA_knn(f5000_coord,10,'distance',os.path.join(asavepath,'f5000_knn10_distance.npz'))
f5000_a['f5000_a_knn20_connectivity']=getA_knn(f5000_coord,20,'connectivity',os.path.join(asavepath,'f5000_knn20_connectivity.npz'))
f5000_a['f5000_a_knn20_distance']=getA_knn(f5000_coord,20,'distance',os.path.join(asavepath,'f5000_knn20_distance.npz'))


(53173, 4)